# Récupérer les métadonnées d'un article pour enregistrer son DOI chez Crossref

Contexte : un article de revue est publié et on souhaite enregistrer son DOI chez CrossRef. Le script python récupère les métadonnées de l'article publié, les ré-organise de sorte à former un fichier XML conforme au schéma CrossRef afin de pouvoir enregistrer le DOI de l'article. Les métadonnées récupérées sont : titre, auteurs, année, DOI, résumé et références biblio. Le fichier XML est ensuite à charger dans [doi.crossref.org](https://doi.crossref.org)

Script utilisé avec des articles publiés par Lodel 1.0 <br /> 
2024-08, Maxence Larrieu

In [1]:
import requests
from  bs4 import BeautifulSoup

In [2]:
## renseigner l'URL de l'article dont il faut enregistrer le DOI
lien_article = "https://revues.mshparisnord.fr/rfim/index.php?id=746"

## 1.  Récupérer les métadonnées depuis la page d'un l'article

In [3]:
## un dictionnaire pour coller les métadonnées
article_md = {}

In [4]:
# Entête de la requête
entete = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
}

In [5]:
## recueillir toutes les données de la page
response = requests.get(lien_article, headers=entete)

## les placer dans Beautiful soup
soup = BeautifulSoup(response.content, "html.parser")

### Récupérer les métadonnées à récupérer

In [6]:
# Titre de l'article
article_md["title"] = soup.find("title").text.strip()

In [7]:
## récupérer tous les auteurs
auth_list = soup.find_all("meta", {"name": "author"})
auths = {}

## passer la liste au niveau unitaire
idx = 0
for item in auth_list :
    full_name = item.get("content").strip()
    auths[idx] = {}
    auths[idx]["given_name"] = full_name[ full_name.index(",") +2 :]
    auths[idx]["surname"] = full_name[ : full_name.index(",")]
    idx += 1

article_md["auths"] = auths

In [8]:
## abstract
article_md["abstract"] = soup.find("meta", {"name" : "DC.Description"}).get("content")

In [9]:
## date
full_date = soup.find("div", {"id" : "date"}).text
article_md["date"] = full_date.split(" ")[1]
if not article_md["date"].isnumeric() : 
    print("//!\\ probleme when getting date")
    quit()

In [10]:
## doi
doi_raw = soup.find("p", {"class" : "article__doi"}).text
doi_raw.split(" ")[1]
article_md["doi"] = doi_raw[ doi_raw.index("doi.org/") + 8 :]


In [11]:
## print article metadatas
for item in article_md : 
    print(item, "\t", article_md[item], "\n")

title 	 Une station de travail audio-numérique open-source pour la plate-forme Web 

auths 	 {0: {'given_name': 'Michel', 'surname': 'Buffa'}, 1: {'given_name': 'Antoine', 'surname': 'Vidal-Mazuy'}} 

abstract 	 Cet article présente WAM Studio (Figure 1), une station de travail audio numérique (DAW) en ligne open source qui tire parti de plusieurs APIs et technologies standards du W3C, telles que Web Audio, WebAssembly, Web Components, Web Midi, Media Devices, etc. WAM Studio s’appuie également sur le standard Web Audio Modules (WAM), qui a été conçu pour faciliter le développement de plugins audio inter-opérables (effets, instruments virtuels, claviers virtuels de piano comme contrôleurs, etc.) sortes de "VSTs pour le Web". Les DAWs sont des logiciels riches en fonctionnalités et donc particulièrement complexes à développer en termes de conception, d’implémentation, de performances et d’ergonomie. Aujourd’hui, la majorité des DAWs en ligne sont commerciaux alors que les seuls exemples

In [12]:
## extra : get biblio
biblio_raw = soup.find_all("p", {"class" : "bibliographie"})
biblio = []
for item in biblio_raw : 
    biblio.append(item.get_text())

biblio[:2]

['Michel Buffa, Shihong Ren, Tom Burns, Antoine Vidal-Mazuy, Stéphane Letz. Evolution of the Web Audio Module Ecosystem. Submitted to the Web Audio Conference 2024, Purdue University, Lafayette, USA, 2024. https://inria.hal.science/hal-04507622/',
 'Buffa M, Vidal-Mazuy A, May L, Winckler M. Wam Studio: A Web-Based Digital Audio Workstation to Empower Cochlear Implant Users. In IFIP Conference on Human-Computer Interaction 2023 Aug 25 (pp.\xa0101-110). Cham: Springer Nature Switzerland. https://inria.hal.science/hal-04233343/']

## Load local XML and parse article metadatas

In [13]:
## charger un fichier XML exemple que l'on modifiera par la suite
with open('crossref-example-to-edit.xml', 'r') as f:
    file = f.read()
    crossref = BeautifulSoup(file, 'html.parser')

C:\Users\Maxence LARRIEU\AppData\Local\Programs\Python\Python312\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [14]:
## xml buffer pour un auteur
xml_author = '''<person_name contributor_role="author" language="en" sequence="first">
<given_name>Malek</given_name>
<surname>Mansour</surname>
</person_name>'''

In [15]:
## ajout des auteurs dans le XML d'exemple

for auth_idx in article_md["auths"] :
    
    # load xml template for one authors
    auth_soup = BeautifulSoup(xml_author, "html.parser")
    ## print(auth_idx)
    
    # change name and surname
    auth_soup.given_name.string.replace_with( article_md["auths"][auth_idx]["given_name"])
    auth_soup.surname.string.replace_with( article_md["auths"][auth_idx]["surname"])
    
    # inclure additionnal as value attribute
    if auth_idx > 0 : 
        pers_name = auth_soup.find("person_name")
        pers_name["sequence"] = "additional"       
    
    # paste data into the XML tree
    crossref.contributors.append(auth_soup)
    

In [16]:
## update doi
crossref.doi.string.replace_with(article_md["doi"])
#crossref.doi

'10.53480/emerg-neurol.5410'

In [17]:
## update ressource link
crossref.resource.string.replace_with(lien_article)

'https://emerging-neurologist.org/article/view/32'

In [26]:
## update title
crossref.title.string.replace_with(article_md["title"])

'Une station de travail audio-numérique open-source pour la plate-forme Web'

In [19]:
for iter_tag in crossref.find_all("year"):
    iter_tag.string.replace_with(article_md["date"])

In [20]:
## update abstract
crossref.find("jats:p").string.replace_with( article_md["abstract"])

'Introduction. Spinal cord involvement as a flare-up of Susac syndrome (SuS) is rarely reported in the literature. The pathogenesis of SuS includes multiple occlusions in micro vessels mediated by an autoimmune response to unknown antigens. This condition is characterized by a triad: central nervous system (CNS) dysfunction with a frequent involvement of the corpus callosum in brain MRI, visual disturbances due to branch retinal artery occlusion, and sensorineural hearing impairment.\r\nCase report. A 50-year-old man presented in December 2019 symptoms associating gait instability, diplopia and hypoacusis. A year later, he developed a vesical-sphincter disorder and a hypoesthesia in his right lower limb. Neurological examination identified a medullary syndrome, cerebellar ataxia and hypoacusis. We performed a brain and spinal MRI showing supratentorial and infratentorial white matter lesions, and gadolinium enhancing cervical lesions. Lumbar puncture with isoelectric focusing results w

In [21]:
## add bibliography

In [22]:
## xml buffer for one ref
xml_ref = '''<citation key="">
<unstructured_citation>one_ref_content</unstructured_citation>
</citation>'''

In [23]:
for idx, ref in enumerate(biblio) : 
    ## laod xml template for one ref
    ref_soup = BeautifulSoup(xml_ref, "html.parser")
    
    ## change ref content
    ref_soup.unstructured_citation.string.replace_with(ref)
    
    ## update idex attribute value
    ref_temp = ref_soup.find("citation")
    ref_temp["key"] = idx
    
    # paste data into the XML tree
    crossref.citation_list.append(ref_soup)

In [24]:
## export XML document

xml_content = str(crossref)

## enregistre le fichier XML sous la forme crossref-auto-273.xml où 273 est l'identifiant de l'URL
with open(f"crossref-auto-{lien_article[-3:]}.xml", "w", encoding="utf-8") as xml_file : 
    xml_file.write(xml_content)

print(f"finished : file crossref-auto-{lien_article[-3:]}.xml is exported")

finished : file crossref-auto-746.xml is exported


In [25]:
xml_file.close()